##### 模型搭建

In [ ]:
import torch
 
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()  # 第一句话，调用父类的构造函数
        self.conv1 = torch.nn.Conv2d(3, 32, 3, 1, 1)
        self.relu1=torch.nn.ReLU()
        self.max_pooling1=torch.nn.MaxPool2d(2,1)
 
        self.conv2 = torch.nn.Conv2d(3, 32, 3, 1, 1)
        self.relu2=torch.nn.ReLU()
        self.max_pooling2=torch.nn.MaxPool2d(2,1)
 
        self.dense1 = torch.nn.Linear(32 * 3 * 3, 128)
        self.dense2 = torch.nn.Linear(128, 10)
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.max_pooling1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.max_pooling2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x
 
model = MyNet()
print(model)
'''运行结果为：
MyNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (max_pooling1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (max_pooling2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (dense1): Linear(in_features=288, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=10, bias=True)
)
'''

##### 一张图片经过FPN网络的变化过程

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = 'configs/detectors/cascade_rcnn_r50_rfp_1x_coco.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/cascade_rcnn_r50_rfp_1x_coco-bbox-448.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0')


In [ ]:

# Use the detector to do inference
img = 'testImages/test2.jpg'
result = inference_detector(model, img)

# Let's plot the result
show_result_pyplot(model, img, result, score_thr=0.3)

In [ ]:
print(l_conv)

In [ ]:
import torch
from mmdet.models.necks.thfpn import THFPN

in_channels = [256, 512, 1024, 2048]
out_channels = 256

#start_level=1,# 开始进行FPN的特征层
#add_extra_convs=True,# 是否需要额外的卷积
num_outs = 5,#输出的特征数

scales = [75, 38, 19, 10]
inputs = [torch.rand(1, c, s, s) # 1, channels, scale, scale
           for c, s in zip(in_channels, scales)] # 将数据打包成元组
self = THFPN(in_channels, 11, len(in_channels)).eval()
#out1 = self.__init__()
#print(len(in_channels))
outputs = self.forward(inputs) # 将数据输入网络进行前向传播
#print(len(outputs))
for i in range(len(outputs)): #输出数据
    print('------------------------')
    #print(f'inchannels[{i}]={in_channels[i]}')
    print(f'outputs[{i}].shape = {outputs[i].shape}')



##### pytorch搭建神经网络

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 构建输入集
x = np.mat('0 0;'
           '0 1;'
           '1 0;'
           '1 1')
x = torch.tensor(x).float()
y = np.mat('1;'
           '0;'
           '0;'
           '1')
y = torch.tensor(y).float()

# 搭建网络
myNet = nn.Sequential(
    nn.Linear(2, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
    nn.Sigmoid()
)
print(myNet)

# 设置优化器
optimzer = torch.optim.SGD(myNet.parameters(), lr=0.05)
loss_func = nn.MSELoss()

for epoch in range(5000):
    out = myNet(x)
    loss = loss_func(out, y)  # 计算误差
    optimzer.zero_grad()  # 清除梯度
    loss.backward()
    optimzer.step()


print(myNet(x).data)

##### Backbone ResNet-34 可视化图像提取过程

In [ ]:
from torch import nn
from torch.nn import functional as F
import torch as t

'''
    实现子Module:ResidualBlock
    ResNet-34结构中都是以2为残差块的
    
    nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        dilation=1,
        groups=1,
        bias=True,
        padding_mode='zeros',)
'''

class ResidualBlock(nn.Module):

    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock, self).__init__()#super().__init__()继承
        self.left=nn.Sequential(#self.left重写
                nn.Conv2d(inchannel, outchannel, 3, stride, 1 , bias=False),
                nn.BatchNorm2d(outchannel),
                nn.ReLU(inplace=True),
                nn.Conv2d(outchannel, outchannel, 3,1,1,bias=False),
                nn.BatchNorm2d(outchannel))
        self.right=shortcut
        
    def forward(self,x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)
    
class ResNet(nn.Module):
    '''
    实现主Module:ResNet34
    ResNet34包含多个layer，每个layer又包含多个reesidal block
    用子module实现residual block，用_make_layer函数实现layer
    
    nn.MaxPool2d(
        kernel_size,
        stride=None,
        padding=0,
        dilation=1,
        return_indices=False,
        ceil_mode=False,)
    '''
    def __init__(self, num_classes = 80):
        super(ResNet,self).__init__()
        #前几层图像转换
        self.pre=nn.Sequential(
                nn.Conv2d(3 ,64 ,7 ,2 ,3 ,bias = False),#输入三通道rgb，输出64通道,7x7size,步长为2,padding为3
                nn.BatchNorm2d(64), # 64通道
                nn.ReLU(inplace=True), # relu 激活
                nn.MaxPool2d(3, 2, 1) #3x3，步长为2，padding为1
                )
        #重复的layer，分别有3,4,6,3个residual block
        self.layer1 = self._make_layer(64,  64 , 3)
        self.layer2 = self._make_layer(64, 128 , 4, stride=2) #默认padding=0
        self.layer3 = self._make_layer(128, 256 , 6, stride=2)
        self.layer4 = self._make_layer(256, 512 , 3, stride=2)
        
        #分类用的全连接
        self.fc=nn.Linear(512, num_classes)#前一个参数是最后神经元的个数最后的num_class是最后的输出类别
        
    def _make_layer(self, inchannel, outchannel, block_num, stride=1):
        '''
        构建layer，包含多个residual
        '''
        shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, 1, stride, bias=False),
                nn.BatchNorm2d(outchannel))
        
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))
        
        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        
        return nn.Sequential(*layers)
    
    def forward(self,x):
        print('图片原始大小',x.size(),)
        
        x=self.pre(x)
        print('conv1:',x.size())
        
        x=self.layer1(x)
        print('conv2:',x.size())
        
        x=self.layer2(x)
        print('conv3:',x.size())
            
        x=self.layer3(x)
        print('conv4:',x.size())
        
        x=self.layer4(x)
        print('conv5:',x.size())
        
        x=F.avg_pool2d(x,7)
        print('池化:',x.size())
        
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        print('全连接:',x.size())
        return x

model = ResNet()
#print(model)
input = t.autograd.Variable(t.randn(1,3,224,224))
o = model(input)
#print(o.size())


In [ ]:
'''
class Person(object):  #class创建类，person 类名，object表示该类从哪里继承下来
    pass
student = Person()   #创建类的实例化
# print(student)
# print(Person)            #实例化变量绑定属性
student.name = "Gavin"   # 为实例变量 student 绑定 name 属性  类似于 赋值 操作
student.score = 100    # 为 其绑定 score 属性
print(student.name)
print(student.score)
'''


#------为更加简单优雅，使用def __init__(self)方法--------
class Person(object):
    def __init__(self, name, score):#本身，属性1，属性2
        self.name = name
        self.score = score
        
        super(Person, self).__init__()#super()方法既能继承父类方法且能重写方法
        self.name1 = name
        assert 3 > 2  #通过调用 abort 来终止！程序运行
        self.score2 = score #上面条件不成立，终止在上面,所以不会得到打印信息
student = Person('Gavin', 100)

print(student.name1)
print(student.score2)




##### 复现resnet-50 fpn pafpn

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F


class Bottleneck(nn.Module):#定义类名bottleneck 从nn.Module继承而来
    expansion = 4 # resnet-50子模块内通道数的倍数关系

    def __init__(self, in_size, size_u, stride=1, is_down=False):#__init__方法，，是否下采样进行通道调整，每一层的第一个卷积
        super(Bottleneck, self).__init__()#super().__init__()方法继承Bottleneck 重写父类
        #-----------------resnet 子模块结构定义CONV1，BN1，CONV2，BN2...---------------------
        self.conv1 = nn.Conv2d(in_size, size_u, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(size_u)

        self.conv2 = nn.Conv2d(size_u, size_u, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(size_u)

        self.conv3 = nn.Conv2d(size_u, size_u * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(size_u * self.expansion)

        self.relu = nn.ReLU(inplace=True)

        self.downsample = nn.Sequential(#下采样卷积定义
            nn.Conv2d(in_size, size_u * self.expansion, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(size_u * self.expansion))
        self.stride = stride

        self.is_down = is_down

    def forward(self, x):#定义子模块内的前向传播函数
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.is_down:#跳连机制 发生在每一层的第一个卷积
            identity = self.downsample(x)

        out += identity#跳连与输出融合
        out = self.relu(out)#输出前激活

        return out #返回最后结果


class Resnt50(nn.Module):#定义每一层的层结构
    def __init__(self):
        super(Resnt50, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.lysize = [64, 128, 256, 512, 1024, 2048]
#resnet-50  4层   //3  4  6  3
        self.layer1 = nn.Sequential(Bottleneck(self.lysize[0], self.lysize[0], 1, True),
                                    Bottleneck(self.lysize[2], self.lysize[0], 1, False),
                                    Bottleneck(self.lysize[2], self.lysize[0], 1, False))

        self.layer2 = nn.Sequential(Bottleneck(self.lysize[2], self.lysize[1], 2, True),
                                    Bottleneck(self.lysize[3], self.lysize[1], 1, False),
                                    Bottleneck(self.lysize[3], self.lysize[1], 1, False),
                                    Bottleneck(self.lysize[3], self.lysize[1], 1, False))

        self.layer3 = nn.Sequential(Bottleneck(self.lysize[3], self.lysize[2], 2, True),
                                    Bottleneck(self.lysize[4], self.lysize[2], 1, False),
                                    Bottleneck(self.lysize[4], self.lysize[2], 1, False),
                                    Bottleneck(self.lysize[4], self.lysize[2], 1, False),
                                    Bottleneck(self.lysize[4], self.lysize[2], 1, False),
                                    Bottleneck(self.lysize[4], self.lysize[2], 1, False))

        self.layer4 = nn.Sequential(Bottleneck(self.lysize[4], self.lysize[3], 2, True),
                                    Bottleneck(self.lysize[5], self.lysize[3], 1, False),
                                    Bottleneck(self.lysize[5], self.lysize[3], 1, False))

        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(self.lysize[5], 3)

        for m in self.modules():#初始化conv,bn模块参数
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):#resnet-50整个模块的前向传播
        conv1 = self.conv1(x)
        bn1 = self.bn1(conv1)
        relu = self.relu(bn1)
        maxpool = self.maxpool(relu)

        layer1 = self.layer1(maxpool)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)
        # x = self.avgpool(layer4)
        # x = x.view(x.shape[0], -1)
        # x = self.fc(x)
        return layer1, layer2, layer3, layer4


class FPN(nn.Module):
    def __init__(self):
        super(FPN, self).__init__()
        self.resnet_feature = Resnt50()# resnet独立的生成每一层的卷积结构
        #层与层之间的卷积结构
        self.conv1 = nn.Conv2d(in_channels=2048, out_channels=256, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(1024, 256, 1, 1, 0)
        self.conv3 = nn.Conv2d(512, 256, 1, 1, 0)
        self.conv4 = nn.Conv2d(256, 256, 1, 1, 0)
        self.fpn_convs = nn.Conv2d(256, 256, 3, 1, 1)#横向连接 调整通道数

    def forward(self, x):#将整个结构的所用组件描述清楚，再将整个组件连接（图像数据的流向）
        layer1, layer2, layer3, layer4 = self.resnet_feature(x)  # channel 256 512 1024 2048，输入x，得到每层特征图，

        P5_1 = self.conv1(layer4)#最高层的特征图，使用1x1卷积提取    backbone级别输出
        P4_1 = self.conv2(layer3)
        P3_1 = self.conv3(layer2)
        P2_1 = self.conv4(layer1)

        size4 = P4_1.shape[2:]#舍去 0  1
        size3 = P3_1.shape[2:]
        size2 = P2_1.shape[2:]

        P5_2 = P5_1
        P4_2 = P4_1 + F.interpolate(P5_2, size=size4, mode='nearest')#FPN降采样融合
        P3_2 = P3_1 + F.interpolate(P4_2, size=size3, mode='nearest')
        P2_2 = P2_1 + F.interpolate(P3_2, size=size2, mode='nearest')

        P5_3 = self.fpn_convs(P5_2)     #FPN网络输出
        P4_3 = self.fpn_convs(P4_2)
        P3_3 = self.fpn_convs(P3_2)
        P2_3 = self.fpn_convs(P2_2)

        return P5_2, P4_2, P3_2, P2_2
        return P2_3, P3_3, P4_3, P5_3


class PAFPN(nn.Module):
    def __init__(self, class_number=80):
        super(PAFPN, self).__init__()
        self.fpn = FPN()#  将 FPN 继承下来 
        self.convN = nn.Conv2d(256, 256, 3, 2, 1)#末端向上融合卷积模块
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        P2_3, P3_3, P4_3, P5_3 = self.fpn(x)#FPN的输出端

        N2 = P2_3  #最下面一级  不涉及特征融合
        N2_1 = self.convN(N2)#向 N3 卷积融合
        N2_1 = self.relu(N2_1)#激活输出

        N3 = N2_1 + P3_3 # 融合

        N3_1 = self.convN(N3)
        N3_1 = self.relu(N3_1)
        N4 = N3_1 + P4_3#融合

        N4_1 = self.convN(N4)
        N4_1 = self.relu(N4_1)
        N5 = N4_1 + P5_3#融合

        return N2, N3, N4, N5
    

class LFP1(nn.Module):
    
    def __init__(self, class_number=80):
        super(LFP1, self).__init__()
        self.pafpn = PAFPN()  #将 PAFPN 继承下来 
        
        self.resnet_feature = Resnt50()# resnet独立的生成每一层的卷积结构
        self.fpn_convs = nn.Conv2d(256, 256, 3, 1, 1)#横向连接 调整通道数
        self.convN = nn.Conv2d(256, 256, 3, 2, 1)#末端向上融合卷积模块
        
        self.convF_1 = nn.Conv2d(256, 256, 1, 1, 0)#通过一个1x1的卷积递归
        self.convF_2 = nn.Conv2d(256, 512, 1, 1, 0)
        self.convF_3 = nn.Conv2d(256, 1024, 1, 1, 0)
        self.convF_4 = nn.Conv2d(256, 2048, 1, 1, 0)
        
        self.conv1 = nn.Conv2d(in_channels=2048, out_channels=256, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(1024, 256, 1, 1, 0)
        self.conv3 = nn.Conv2d(512, 256, 1, 1, 0)
        self.conv4 = nn.Conv2d(256, 256, 1, 1, 0)
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        
        #------------第一次fpn
        layer1, layer2, layer3, layer4 = self.resnet_feature(x)  # channel 256 512 1024 2048，输入x，得到每层特征图，

        P5_1 = self.conv1(layer4)#最高层的特征图，使用1x1卷积提取    backbone级别输出
        P4_1 = self.conv2(layer3)
        P3_1 = self.conv3(layer2)
        P2_1 = self.conv4(layer1)

        size4 = P4_1.shape[2:]#舍去 0  1
        size3 = P3_1.shape[2:]
        size2 = P2_1.shape[2:]

        P5_2 = P5_1
        P4_2 = P4_1 + F.interpolate(P5_2, size=size4, mode='nearest')#FPN降采样融合
        P3_2 = P3_1 + F.interpolate(P4_2, size=size3, mode='nearest')
        P2_2 = P2_1 + F.interpolate(P3_2, size=size2, mode='nearest')

        #单层递归
        F2_1 = self.convF_1(P2_2)
        F2_1 = self.relu(F2_1)
        layer1 = F2_1 + layer1
        
        F3_1 = self.convF_2(P3_2)
        F3_1 = self.relu(F3_1)
        layer2 = F3_1 + layer2
        
        F4_1 = self.convF_3(P4_2)
        F4_1 = self.relu(F4_1)  
        layer3 = F4_1 + layer3
        
        F5_1 = self.convF_4(P5_2)
        F5_1 = self.relu(F5_1)
        layer4 = F5_1 + layer4
        
        #-------------------第二次FPN
        P5_1 = self.conv1(layer4)#最高层的特征图，使用1x1卷积提取    backbone级别输出
        P4_1 = self.conv2(layer3)
        P3_1 = self.conv3(layer2)
        P2_1 = self.conv4(layer1)

        size4 = P4_1.shape[2:]#舍去 0  1
        size3 = P3_1.shape[2:]
        size2 = P2_1.shape[2:]

        P5_2 = P5_1
        P4_2 = P4_1 + F.interpolate(P5_2, size=size4, mode='nearest')#FPN降采样融合
        P3_2 = P3_1 + F.interpolate(P4_2, size=size3, mode='nearest')
        P2_2 = P2_1 + F.interpolate(P3_2, size=size2, mode='nearest')
        
        P5_3 = self.fpn_convs(P5_2)     #FPN网络输出
        P4_3 = self.fpn_convs(P4_2)
        P3_3 = self.fpn_convs(P3_2)
        P2_3 = self.fpn_convs(P2_2)
        
        #------pafpn------
        N2 = P2_3  #最下面一级  不涉及特征融合
        N2_1 = self.convN(N2)#向 N3 卷积融合
        N2_1 = self.relu(N2_1)#激活输出

        N3 = N2_1 + P3_3 # 融合

        N3_1 = self.convN(N3)
        N3_1 = self.relu(N3_1)
        N4 = N3_1 + P4_3#融合

        N4_1 = self.convN(N4)
        N4_1 = self.relu(N4_1)
        N5 = N4_1 + P5_3#融合

        return N2, N3, N4, N5

    
class LFP(nn.Module):
    def __init__(self, class_number=80):
        super(LFP, self).__init__()
        self.pafpn = PAFPN()
        #==================FPN=======================
        self.resnet_feature = Resnt50()# resnet独立的生成每一层的卷积结构
        #层与层之间的卷积结构
        self.conv1 = nn.Conv2d(in_channels=2048, out_channels=256, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(1024, 256, 1, 1, 0)
        self.conv3 = nn.Conv2d(512, 256, 1, 1, 0)
        self.conv4 = nn.Conv2d(256, 256, 1, 1, 0)
        self.fpn_convs = nn.Conv2d(256, 256, 3, 1, 1)#横向连接 调整通道数
        self.convN = nn.Conv2d(256, 256, 3, 2, 1)#末端向上融合卷积模块
        self.convL = nn.Conv2d(256, 256, 1, 1, 0)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):#将整个结构的所用组件描述清楚，再将整个组件连接（图像数据的流向）
        #————————>FPN————————>PAFPN输出
        layer1, layer2, layer3, layer4 = self.resnet_feature(x)  # channel 256 512 1024 2048，输入x，得到每层特征图，

        P5_1 = self.conv1(layer4)#最高层的特征图，使用1x1卷积提取    backbone级别输出
        P4_1 = self.conv2(layer3)
        P3_1 = self.conv3(layer2)
        P2_1 = self.conv4(layer1)
        print(P2_1.shape)

        size4 = P4_1.shape[2:]#舍去 0  1
        size3 = P3_1.shape[2:]
        size2 = P2_1.shape[2:]

        P5_2 = P5_1
        P4_2 = P4_1 + F.interpolate(P5_2, size=size4, mode='nearest')#FPN降采样融合
        P3_2 = P3_1 + F.interpolate(P4_2, size=size3, mode='nearest')
        P2_2 = P2_1 + F.interpolate(P3_2, size=size2, mode='nearest')

        P5_3 = self.fpn_convs(P5_2)     #FPN网络输出
        P4_3 = self.fpn_convs(P4_2)
        P3_3 = self.fpn_convs(P3_2)
        P2_3 = self.fpn_convs(P2_2)
        
        #==================PAFPN=========================
        N2 = P2_3  #最下面一级  不涉及特征融合
        N2_1 = self.convN(N2)#向 N3 卷积融合
        N2_1 = self.relu(N2_1)#激活输出

        N3 = N2_1 + P3_3 # 融合

        N3_1 = self.convN(N3)
        N3_1 = self.relu(N3_1)
        N4 = N3_1 + P4_3#融合

        N4_1 = self.convN(N4)
        N4_1 = self.relu(N4_1)
        N5 = N4_1 + P5_3#融合
        
 
    
        #===================LFP=====================
        N5_1 = self.convL(N5)
        N5_1 = self.relu(N5_1)
        print("打印N5_1：",N5.shape)
        print("打印P5_1：",P5_1.shape)
 
        #-----LFP循环 更新FPN参数------------
        P5_2 = N5_1 + P5_1 #改进点，引入反馈
        P4_2 = P4_1 + F.interpolate(P5_2, size=size4, mode='nearest')#FPN降采样融合
        P3_2 = P3_1 + F.interpolate(P4_2, size=size3, mode='nearest')
        P2_2 = P2_1 + F.interpolate(P3_2, size=size2, mode='nearest')
        
        P5_3 = self.fpn_convs(P5_2)     #FPN网络输出
        P4_3 = self.fpn_convs(P4_2)
        P3_3 = self.fpn_convs(P3_2)
        P2_3 = self.fpn_convs(P2_2)
        
        N2 = P2_3  #最下面一级  不涉及特征融合
        N2_1 = self.convN(N2)#向 N3 卷积融合
        N2_1 = self.relu(N2_1)#激活输出

        N3 = N2_1 + P3_3 # 融合

        N3_1 = self.convN(N3)
        N3_1 = self.relu(N3_1)
        N4 = N3_1 + P4_3#融合

        N4_1 = self.convN(N4)
        N4_1 = self.relu(N4_1)
        N5 = N4_1 + P5_3#融合
        
        return N2, N3, N4, N5

if __name__ == '__main__':
    
    from torchsummary import summary

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LFP().to(device)

    summary(model, (3, 600, 600))#输入数据 3x600x512
    '''
    for parameters in model.parameters():#打印出参数矩阵及值
        print(parameters)
    '''
    
    '''
    for name, parameters in model.named_parameters():#打印出每一层的参数的大小
        print(name, ':', parameters.size())
    '''

    
    


In [ ]:
import pytest
import torch
from torch.nn.modules.batchnorm import _BatchNorm

from mmdet.models.necks import FPN, ChannelMapper


def test_fpn():
    """Tests fpn. 每一层对应的特征图大小以及C234依然混淆    """
    s = 600
    in_channels = [256, 512, 1024, 2048]
    feat_sizes = [s // 2**i for i in range(4)]  # [600, 300, 150, 75]
    #print(feat_sizes)
    out_channels = 256
    # `num_outs` is not equal to len(in_channels) - start_level
    with pytest.raises(AssertionError):
        FPN(in_channels=in_channels,
            out_channels=out_channels,
            start_level=1,
            num_outs=2)

    # `end_level` is larger than len(in_channels) - 1
    with pytest.raises(AssertionError):
        FPN(in_channels=in_channels,
            out_channels=out_channels,
            start_level=1,
            end_level=4,
            num_outs=2)

    # `num_outs` is not equal to end_level - start_level
    with pytest.raises(AssertionError):
        FPN(in_channels=in_channels,
            out_channels=out_channels,
            start_level=1,
            end_level=3,
            num_outs=1)

    # Invalid `add_extra_convs` option
    with pytest.raises(AssertionError):
        FPN(in_channels=in_channels,
            out_channels=out_channels,
            start_level=1,
            add_extra_convs='on_xxx',
            num_outs=5)

    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        start_level=1,
        add_extra_convs=True,#添加了额外的卷积操作时，如 RetinaNet
        num_outs=5)
    #print(fpn_model)#输出卷积信息，lateral_conv和fpn_conv信息

    # FPN expects a multiple levels of features per image
    feats = [
        torch.rand(1, in_channels[i], feat_sizes[i], feat_sizes[i])
        for i in range(len(in_channels))
    ]
    outs = fpn_model(feats)
    assert fpn_model.add_extra_convs == 'on_input'
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # Tests for fpn with no extra convs (pooling is used instead)
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        start_level=1,
        add_extra_convs=False,#没有添加额外的卷积操作，如 Fasetr R-CNN
        num_outs=5)
    outs = fpn_model(feats)
    #print(outs)
    
    assert len(outs) == fpn_model.num_outs
    assert not fpn_model.add_extra_convs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # Tests for fpn with lateral bns
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        start_level=1,
        add_extra_convs=True,
        no_norm_on_lateral=False,
        norm_cfg=dict(type='BN', requires_grad=True),
        num_outs=5)
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    assert fpn_model.add_extra_convs == 'on_input'
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)
    bn_exist = False
    for m in fpn_model.modules():
        if isinstance(m, _BatchNorm):
            bn_exist = True
    assert bn_exist

    # Bilinear upsample
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        start_level=1,
        add_extra_convs=True,
        upsample_cfg=dict(mode='bilinear', align_corners=True),
        num_outs=5)
    fpn_model(feats)
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    assert fpn_model.add_extra_convs == 'on_input'
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # Scale factor instead of fixed upsample size upsample
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        start_level=1,
        add_extra_convs=True,
        upsample_cfg=dict(scale_factor=2),
        num_outs=5)
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # Extra convs source is 'inputs'
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        add_extra_convs='on_input',
        start_level=1,
        num_outs=5)
    assert fpn_model.add_extra_convs == 'on_input'
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # Extra convs source is 'laterals'
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        add_extra_convs='on_lateral',
        start_level=1,
        num_outs=5)
    assert fpn_model.add_extra_convs == 'on_lateral'
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # Extra convs source is 'outputs'
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        add_extra_convs='on_output',
        start_level=1,
        num_outs=5)
    assert fpn_model.add_extra_convs == 'on_output'
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # extra_convs_on_inputs=False is equal to extra convs source is 'on_output'
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        add_extra_convs=True,
        extra_convs_on_inputs=False,
        start_level=1,
        num_outs=5,
    )
    assert fpn_model.add_extra_convs == 'on_output'
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)

    # extra_convs_on_inputs=True is equal to extra convs source is 'on_input'
    fpn_model = FPN(
        in_channels=in_channels,
        out_channels=out_channels,
        add_extra_convs=True,
        extra_convs_on_inputs=True,
        start_level=1,
        num_outs=5,
    )
    assert fpn_model.add_extra_convs == 'on_input'
    outs = fpn_model(feats)
    assert len(outs) == fpn_model.num_outs
    for i in range(fpn_model.num_outs):
        outs[i].shape[1] == out_channels
        outs[i].shape[2] == outs[i].shape[3] == s // (2**i)


In [ ]:
import pytest
import torch
from torch.nn.modules.batchnorm import _BatchNorm
from mmdet.models.necks import PAFPN,ChannelMapper

if __name__ == '__main__':
    s = 600
    in_channels = [256, 512, 1024, 2048]
    feat_sizes = [s // 2**i for i in range(4)]  # [600, 300, 150, 75]
#    print(feat_sizes)
    out_channels = 256
    fpn_model = PAFPN(
        in_channels = in_channels,
        out_channels = out_channels,
        start_level = 0,
        add_extra_convs = False,#C2---C5  用mask-rcnn
        num_outs = 5)
    '''
        feats = [
        torch.rand(1, in_channels[i], feat_sizes[i], feat_sizes[i])
        for i in range(len(in_channels))
            ]
    '''


    print(fpn_model)

In [ ]:
from mmcv.runner import auto_fp16

from mmdet.models.builder import NECKS
from mmdet.models.necks.fpn import FPN


#@NECKS.register_module()
class LFP(FPN):

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_outs,
                 start_level=0,
                 end_level=-1,
                 add_extra_convs=False,
                 extra_convs_on_inputs=True,
                 relu_before_extra_convs=False,
                 no_norm_on_lateral=False,
                 conv_cfg=None,
                 norm_cfg=None,
                 act_cfg=None):
        super(LFP,
              self).__init__(in_channels, out_channels, num_outs, start_level,
                             end_level, add_extra_convs, extra_convs_on_inputs,
                             relu_before_extra_convs, no_norm_on_lateral,
                             conv_cfg, norm_cfg, act_cfg)

        # add extra bottom up pathway（至下而上的卷积融合）
#        self.recursive_convs = nn.ModuleList()#递归卷积
        self.downsample_convs = nn.ModuleList()#下采样卷积，图中的后半部分
        self.pafpn_convs = nn.ModuleList()#pafpn卷积

        self.convL = nn.Conv2d(256, 256, 1, 1, 0)

#----------------------------------------------
        for i in range(self.start_level + 1, self.backbone_end_level):

            d_conv = ConvModule(
                out_channels,                         
                out_channels,
                3,
                stride=2,
                padding=1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=act_cfg,
                inplace=False)

            pafpn_conv = ConvModule(       #输出前卷积
                out_channels,
                out_channels,
                3,
                padding=1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=act_cfg,
                inplace=False)

            self.downsample_convs.append(d_conv)
            self.pafpn_convs.append(pafpn_conv)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from mmcv.cnn import ConvModule
from mmcv.runner import auto_fp16

from mmdet.models.builder import NECKS
from mmdet.models.necks.fpn import FPN


#@NECKS.register_module()
class LFP(FPN):

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_outs,
                 start_level=0,
                 end_level=-1,
                 add_extra_convs=False,
                 extra_convs_on_inputs=True,
                 relu_before_extra_convs=False,
                 no_norm_on_lateral=False,
                 conv_cfg=None,
                 norm_cfg=None,
                 act_cfg=None):
        super(LFP,
              self).__init__(in_channels, out_channels, num_outs, start_level,
                             end_level, add_extra_convs, extra_convs_on_inputs,
                             relu_before_extra_convs, no_norm_on_lateral,
                             conv_cfg, norm_cfg, act_cfg)

        # add extra bottom up pathway（至下而上的卷积融合）
#        self.recursive_convs = nn.ModuleList()#递归卷积
        self.downsample_convs = nn.ModuleList()#下采样卷积，图中的后半部分
        self.pafpn_convs = nn.ModuleList()#pafpn卷积

        self.convL = nn.Conv2d(256, 256, 1, 1, 0)
#----------------------------------------------
        for i in range(self.start_level + 1, self.backbone_end_level):

            d_conv = ConvModule(         #降采样卷积 k=3,p=1,s=2时的卷积输入输出大小/2
                out_channels,                         
                out_channels,
                3,
                stride=2,
                padding=1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=act_cfg,
                inplace=False)

            pafpn_conv = ConvModule(       #输出前卷积 k=3,p=1,s=1时的卷积输入输出大小不变
                out_channels,
                out_channels,
                3,
                padding=1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=act_cfg,
                inplace=False)

            self.downsample_convs.append(d_conv)
            self.pafpn_convs.append(pafpn_conv)


    @auto_fp16()
    def forward(self, inputs):
        """Forward function."""
        assert len(inputs) == len(self.in_channels)
        # build laterals    调整C=256通道后
        laterals = [
            lateral_conv(inputs[i + self.start_level])
            for i, lateral_conv in enumerate(self.lateral_convs)
        ]
        pre_laterals = [
            lateral_conv(inputs[i + self.start_level])
            for i, lateral_conv in enumerate(self.lateral_convs)
        ]

#C
        # build top-down path
        used_backbone_levels = len(laterals) # used_backbone_levels = 4
#        print("used_backbone_levels：",len(laterals))
#M
  #融合前后存在覆盖问题
        for i in range(used_backbone_levels - 1, 0, -1):# 倒序   3，2，1(倒序输出，不包括0)
            prev_shape = laterals[i - 1].shape[2:] #上采样过程中间的一个size参数
            #laterals[i]上一层的下采样与backbone中laterals卷积输出的lateral[i-1]进行相加融合，融合为lateral[i-1]
            laterals[i - 1] = laterals[i - 1] + F.interpolate(laterals[i],
                size=prev_shape, mode='nearest')#最近领域插值
            #保留前次循环的特征图
            pre_laterals[i - 1] = laterals[i - 1]#初始化
        #print(f'里面laterals[{i}]={laterals[i-1].shape}')
            #    print(f'outputs[{i}].shape = {outputs[i].shape}')  带编号i输出
        # build outputs
        # part 1: from original levels
        inter_outs = [
            self.fpn_convs[i](laterals[i]) for i in range(used_backbone_levels)
        ]   #第一级输出，后面融合的使用append()方法
        #定义pre
        pre_inter_outs = inter_outs
        # part 2: add bottom-up path  至下而上路径融合输出
        for i in range(0, used_backbone_levels - 1):
            inter_outs[i + 1] = inter_outs[i + 1] + self.downsample_convs[i](inter_outs[i])

            pre_inter_outs[i + 1] = inter_outs[i + 1]#初始化




#此时的输出是：在RetinaNet中C3，C4，C5在backbone输出，FPN融合至下而上最后输出
#========================Improved===============

        
        loop = 2
        for i in range(loop):#概括整个循环
            laterals[used_backbone_levels - 1] = laterals[used_backbone_levels - 1] + self.convL(inter_outs[used_backbone_levels - 1])
#        print("laterals[used_backbone_levels - 1].shape:",laterals[used_backbone_levels - 1].shape)
#C
            laterals = [
            lateral_conv(inputs[i + self.start_level])
            for i, lateral_conv in enumerate(self.lateral_convs)
            ]       


#C ---> LFP function, here,forward again        
            laterals[used_backbone_levels - 1] = laterals[used_backbone_levels - 1] + self.convL(inter_outs[used_backbone_levels - 1])
            pre_laterals[used_backbone_levels - 1] = laterals[used_backbone_levels - 1]#最高层保留
        # build top-down path
            used_backbone_levels = len(laterals) # used_backbone_levels = 4
#M
            for i in range(used_backbone_levels - 1, 0, -1):# 倒序   3，2，1
                prev_shape = laterals[i - 1].shape[2:]
                laterals[i - 1] = laterals[i - 1] + F.interpolate(
                    laterals[i], size=prev_shape, mode='nearest') + pre_laterals[i - 1] + F.interpolate(inter_outs[i],
                size=prev_shape, mode='nearest')   #设置空洞卷积
                pre_laterals[i - 1] = laterals[i - 1]
     
            #laterals[2],   laterals[1],   laterals[0] 对应 M4，M3，M2
        # build outputs
        # part 1: from original levels
            inter_outs = [  #for i range(3)  输出0,1，2不包括3
                    self.fpn_convs[i](laterals[i]) for i in range(used_backbone_levels)
                ]   #第一级输出，后面融合的使用append()方法
#P
        # part 2: add bottom-up path  至下而上路径融合输出
            for i in range(0, used_backbone_levels - 1): # 0,1,2 不包括3
                inter_outs[i + 1] = inter_outs[i + 1] + self.downsample_convs[i](inter_outs[i]) + pre_inter_outs[i + 1]
                pre_inter_outs[i + 1] = inter_outs[i + 1]

 #=================================================================




 
        outs = []
        #inter_outs[0]不涉及融合，将inter_outs[0]粘贴进入元组
        outs.append(inter_outs[0])#融合第一级输出
        outs.extend([
            self.pafpn_convs[i - 1](inter_outs[i])
            for i in range(1, used_backbone_levels)#1，2，3
        ])

        # part 3: add extra levels   推理P6，P7层的输出
        if self.num_outs > len(outs):#如果设定的输出 大于 backbone的输出
            # use max pool to get more levels on top of outputs
            # (e.g., Faster R-CNN, Mask R-CNN)
            if not self.add_extra_convs:#如果没有额外添加其它层，C2-C5，faser，mask
                for i in range(self.num_outs - used_backbone_levels):
                    outs.append(F.max_pool2d(outs[-1], 1, stride=2))

            # add conv layers on top of original feature maps (RetinaNet)
            else:
                if self.extra_convs_on_inputs:
                    orig = inputs[self.backbone_end_level - 1]
                    outs.append(self.fpn_convs[used_backbone_levels](orig))
                else:
                    outs.append(self.fpn_convs[used_backbone_levels](outs[-1]))
                for i in range(used_backbone_levels + 1, self.num_outs):
                    if self.relu_before_extra_convs:
                        outs.append(self.fpn_convs[i](F.relu(outs[-1])))
                    else:
                        outs.append(self.fpn_convs[i](outs[-1]))
        return tuple(outs)


In [ ]:


import torch
from torchsummary import summary
    
if __name__ == '__main__':
    s = 300
    in_channels = [256, 512, 1024, 2048]
    feat_sizes = [s // 2**i for i in range(4)]  # [600, 300, 150, 75]
#    print(feat_sizes)
    out_channels = 256
    num_outs = 5
    fpn_model = LFP(
        in_channels = in_channels,
        out_channels = out_channels,
        start_level = 0,
        add_extra_convs = False,#C2---C5  用mask-rcnn
        num_outs = num_outs )
    
    inputs = [
        torch.rand(1, in_channels[i], feat_sizes[i], feat_sizes[i])
        for i in range(len(in_channels))]
      
    #print(len(inputs))
    LFP.forward(fpn_model, inputs)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #summary(fpn_model, (3, 600, 600))#输入数据 3x600x512
    #print(out)


In [1]:
import torch.nn as nn
import torch.nn.functional as F
from mmcv.cnn import ConvModule
from mmcv.runner import auto_fp16

from mmdet.models.builder import NECKS
from mmdet.models.necks.fpn import FPN


from mmcv.cnn import ConvModule, build_upsample_layer, xavier_init
from mmcv.ops.carafe import CARAFEPack



#@NECKS.register_module()
class LFP(nn.Module):
    """FPN_CARAFE is a more flexible implementation of FPN. It allows more
    choice for upsample methods during the top-down pathway.

    It can reproduce the preformance of ICCV 2019 paper
    CARAFE: Content-Aware ReAssembly of FEatures
    Please refer to https://arxiv.org/abs/1905.02188 for more details.

    Args:
        in_channels (list[int]): Number of channels for each input feature map.
        out_channels (int): Output channels of feature pyramids.
        num_outs (int): Number of output stages.
        start_level (int): Start level of feature pyramids.
            (Default: 0)
        end_level (int): End level of feature pyramids.
            (Default: -1 indicates the last level).
        norm_cfg (dict): Dictionary to construct and config norm layer.
        activate (str): Type of activation function in ConvModule
            (Default: None indicates w/o activation).
        order (dict): Order of components in ConvModule.
        upsample (str): Type of upsample layer.
        upsample_cfg (dict): Dictionary to construct and config upsample layer.
    """

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_outs,
                 start_level=0,
                 end_level=-1,
                 norm_cfg=None,
                 act_cfg=None,
                 order=('conv', 'norm', 'act'),
                 upsample_cfg=dict(
                     type='carafe',
                     up_kernel=5,
                     up_group=1,
                     encoder_kernel=3,
                     encoder_dilation=1)):
        super(LFP, self).__init__()
        assert isinstance(in_channels, list)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_ins = len(in_channels)
        self.num_outs = num_outs
        self.norm_cfg = norm_cfg
        self.act_cfg = act_cfg
        self.with_bias = norm_cfg is None
        self.upsample_cfg = upsample_cfg.copy()
        self.upsample = self.upsample_cfg.get('type')
        self.relu = nn.ReLU(inplace=False)

        self.order = order
        assert order in [('conv', 'norm', 'act'), ('act', 'conv', 'norm')]

        assert self.upsample in [
            'nearest', 'bilinear', 'deconv', 'pixel_shuffle', 'carafe', None
        ]
        if self.upsample in ['deconv', 'pixel_shuffle']:
            assert hasattr(
                self.upsample_cfg,
                'upsample_kernel') and self.upsample_cfg.upsample_kernel > 0
            self.upsample_kernel = self.upsample_cfg.pop('upsample_kernel')

        if end_level == -1:
            self.backbone_end_level = self.num_ins
            assert num_outs >= self.num_ins - start_level
        else:
            # if end_level < inputs, no extra level is allowed
            self.backbone_end_level = end_level
            assert end_level <= len(in_channels)
            assert num_outs == end_level - start_level
        self.start_level = start_level
        self.end_level = end_level

        self.lateral_convs = nn.ModuleList()
        self.fpn_convs = nn.ModuleList()
        self.upsample_modules = nn.ModuleList()

#add=========================================
        # add extra bottom up pathway（至下而上的卷积融合）
        self.downsample_convs = nn.ModuleList()#下采样卷积，图中的后半部分
        self.pafpn_convs = nn.ModuleList()#至下而上pafpn卷积
        for i in range(self.start_level + 1, self.backbone_end_level):
            d_conv = ConvModule(
                out_channels,                         
                out_channels,
                3,
                stride=2,
                padding=1,
                norm_cfg=self.norm_cfg,
                bias=self.with_bias,
                act_cfg=act_cfg,
                inplace=False,
                order=self.order)

            pafpn_conv = ConvModule(      #输出前卷积
                out_channels,
                out_channels,
                3,
                padding=1,
                norm_cfg=self.norm_cfg,
                bias=self.with_bias,
                act_cfg=act_cfg,
                inplace=False,
                order=self.order)
            self.downsample_convs.append(d_conv)
            self.pafpn_convs.append(pafpn_conv)
#=========================================

        for i in range(self.start_level, self.backbone_end_level):
            l_conv = ConvModule(
                in_channels[i],
                out_channels,
                1,
                norm_cfg=norm_cfg,
                bias=self.with_bias,
                act_cfg=act_cfg,
                inplace=False,
                order=self.order)
            fpn_conv = ConvModule(
                out_channels,
                out_channels,
                3,
                padding=1,
                norm_cfg=self.norm_cfg,
                bias=self.with_bias,
                act_cfg=act_cfg,
                inplace=False,
                order=self.order)
            if i != self.backbone_end_level - 1:
                upsample_cfg_ = self.upsample_cfg.copy()
                if self.upsample == 'deconv':
                    upsample_cfg_.update(
                        in_channels=out_channels,
                        out_channels=out_channels,
                        kernel_size=self.upsample_kernel,
                        stride=2,
                        padding=(self.upsample_kernel - 1) // 2,
                        output_padding=(self.upsample_kernel - 1) // 2)
                elif self.upsample == 'pixel_shuffle':
                    upsample_cfg_.update(
                        in_channels=out_channels,
                        out_channels=out_channels,
                        scale_factor=2,
                        upsample_kernel=self.upsample_kernel)
                elif self.upsample == 'carafe':
                    upsample_cfg_.update(channels=out_channels, scale_factor=2)
                else:
                    # suppress warnings
                    align_corners = (None
                                     if self.upsample == 'nearest' else False)
                    upsample_cfg_.update(
                        scale_factor=2,
                        mode=self.upsample,
                        align_corners=align_corners)
                upsample_module = build_upsample_layer(upsample_cfg_)
                self.upsample_modules.append(upsample_module)
            self.lateral_convs.append(l_conv)
            self.fpn_convs.append(fpn_conv)

        # add extra conv layers (e.g., RetinaNet)
        extra_out_levels = (
            num_outs - self.backbone_end_level + self.start_level)
        if extra_out_levels >= 1:
            for i in range(extra_out_levels):
                in_channels = (
                    self.in_channels[self.backbone_end_level -
                                     1] if i == 0 else out_channels)
                extra_l_conv = ConvModule(
                    in_channels,
                    out_channels,
                    3,
                    stride=2,
                    padding=1,
                    norm_cfg=norm_cfg,
                    bias=self.with_bias,
                    act_cfg=act_cfg,
                    inplace=False,
                    order=self.order)
                if self.upsample == 'deconv':
                    upsampler_cfg_ = dict(
                        in_channels=out_channels,
                        out_channels=out_channels,
                        kernel_size=self.upsample_kernel,
                        stride=2,
                        padding=(self.upsample_kernel - 1) // 2,
                        output_padding=(self.upsample_kernel - 1) // 2)
                elif self.upsample == 'pixel_shuffle':
                    upsampler_cfg_ = dict(
                        in_channels=out_channels,
                        out_channels=out_channels,
                        scale_factor=2,
                        upsample_kernel=self.upsample_kernel)
                elif self.upsample == 'carafe':
                    upsampler_cfg_ = dict(
                        channels=out_channels,
                        scale_factor=2,
                        **self.upsample_cfg)
                else:
                    # suppress warnings
                    align_corners = (None
                                     if self.upsample == 'nearest' else False)
                    upsampler_cfg_ = dict(
                        scale_factor=2,
                        mode=self.upsample,
                        align_corners=align_corners)
                upsampler_cfg_['type'] = self.upsample
                upsample_module = build_upsample_layer(upsampler_cfg_)
                extra_fpn_conv = ConvModule(
                    out_channels,
                    out_channels,
                    3,
                    padding=1,
                    norm_cfg=self.norm_cfg,
                    bias=self.with_bias,
                    act_cfg=act_cfg,
                    inplace=False,
                    order=self.order)
                self.upsample_modules.append(upsample_module)
                self.fpn_convs.append(extra_fpn_conv)
                self.lateral_convs.append(extra_l_conv)

    # default init_weights for conv(msra) and norm in ConvModule
    def init_weights(self):
        """Initialize the weights of module."""
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                xavier_init(m, distribution='uniform')
        for m in self.modules():
            if isinstance(m, CARAFEPack):
                m.init_weights()

    def slice_as(self, src, dst):
        """Slice ``src`` as ``dst``

        Note:
            ``src`` should have the same or larger size than ``dst``.

        Args:
            src (torch.Tensor): Tensors to be sliced.
            dst (torch.Tensor): ``src`` will be sliced to have the same
                size as ``dst``.

        Returns:
            torch.Tensor: Sliced tensor.
        """
        assert (src.size(2) >= dst.size(2)) and (src.size(3) >= dst.size(3))
        if src.size(2) == dst.size(2) and src.size(3) == dst.size(3):
            return src
        else:
            return src[:, :, :dst.size(2), :dst.size(3)]

    def tensor_add(self, a, b):
        """Add tensors ``a`` and ``b`` that might have different sizes."""
        if a.size() == b.size():
            c = a + b
        else:
            c = a + self.slice_as(b, a)
        return c

    def forward(self, inputs):
        """Forward function."""
        assert len(inputs) == len(self.in_channels)

        # build laterals
        laterals = []
        for i, lateral_conv in enumerate(self.lateral_convs):
            if i <= self.backbone_end_level - self.start_level:
                input = inputs[min(i + self.start_level, len(inputs) - 1)]
            else:
                input = laterals[-1]
            lateral = lateral_conv(input)
            laterals.append(lateral)

        # build top-down path
        for i in range(len(laterals) - 1, 0, -1):
            if self.upsample is not None:
                upsample_feat = self.upsample_modules[i - 1](laterals[i])
            else:
                upsample_feat = laterals[i]
            laterals[i - 1] = self.tensor_add(laterals[i - 1], upsample_feat)
#===============================improved=============================
        used_backbone_levels = len(laterals)
        inter_outs = [
            self.fpn_convs[i](laterals[i]) for i in range(used_backbone_levels)
        ]   #第一级输出，后面融合的使用append()方法

        # add bottom-up path
        #for i in range(start_level + 1, used_backbone_levels):#1，2，3
            #inter_outs[i] = inter_outs[i] + self.downsample_convs[i-1](inter_outs[i-1])

#====================================================================


        outs = []
        #inter_outs[0]不涉及融合，将inter_outs[0]粘贴进入元组
        outs.append(inter_outs[0])#融合第一级输出
        outs.extend([
            self.pafpn_convs[i - 1](inter_outs[i])
            for i in range(1, used_backbone_levels)#1，2，3
        ])

        return tuple(outs)

    
    


import torch
from torchsummary import summary
    
if __name__ == '__main__':
    s = 300
    in_channels = [256, 512, 1024, 2048]
    feat_sizes = [s // 2**i for i in range(4)]  # [600, 300, 150, 75]
#    print(feat_sizes)
    out_channels = 256
    num_outs = 5
    fpn_model = LFP(
        in_channels = in_channels,
        out_channels = out_channels,
        start_level = 0,
        num_outs = num_outs)
    
    inputs = [
        torch.rand(1, in_channels[i], feat_sizes[i], feat_sizes[i])
        for i in range(len(in_channels))]
      
    #print(len(inputs))
    LFP.forward(fpn_model, inputs)
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #summary(fpn_model, (3, 600, 600))#输入数据 3x600x512
    #print(out)


/home/htang/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmcv/utils/registry.py:64: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


RuntimeError: Carafe is not implemented on CPU